# ML Assignment 6

Train a Gaussian NBC with the EM algorithm. 
Compare results with those of K-means clustering provided in Sklearn.

The E-M algorithm:
1. Compute the expected outcome for each example/sample given estimates for priors and distribution.
2. Compute new estimates for priors and distributions bases on the estimated expected values for how much each sample belongs to the respective distribution.

In [1]:
%cd C:\Users\Viktor\PycharmProjects\ML_Assignment5

import numpy as np
from sklearn import datasets, metrics
from numpy.linalg import norm

C:\Users\Viktor\PycharmProjects\ML_Assignment5


# 1. Load and split the digits dataset


In [2]:
#Load and split the digit dataset

digit_dataset = datasets.load_digits()

digits_x = digit_dataset.data
digits_y = digit_dataset.target

num_examples = len(digits_x)
num_split = int(0.7*num_examples)

#Split into training and test data
digits_train_features = digits_x[:num_split]
digits_train_labels =  digits_y[:num_split]
digits_test_features = digits_x[num_split:]
digits_test_labels = digits_y[num_split:]

# 2. Implementation of the E-M algorithm


In [3]:
class ExpectationMaximisation:
    
    def __init__(self, features, k = 10):
        
        #Initialise pi_k, mu_k, sum_k where pi_k is class prior, mu_k is means for attribute value j in class k
        self.CovK = 1
        self.means = 1
        self.pi_k = np.ones([10,1])*1/10
        self.clustered = []

        #Make random subsets for initialization
        number_of_samples = len(features)
        subset_size = number_of_samples/k
        
        for i in range(k):
            self.clustered.append((features[round(i*subset_size):round((i+1)*subset_size)]))  
        
        #Find the number of unique values in features. 17 for digits, 3 for sumdigits
        uniqueval = 0
        uniquelist = []
        for pic in features:
            for pixel in pic:
                if pixel not in uniquelist:
                    uniquelist.append(pixel)
        uniqueval = len(uniquelist)
        
        attributesums = np.ones([k,len(features[0])])*0.1
        clustercount = np.zeros(k)
        self.means = np.zeros([k,len(features[0])])  # Means for all features in cluster k
        
        valuecountmatrix = np.zeros([k,uniqueval,len(features[1])])
        for k,cluster in enumerate(self.clustered): #Iterate through clusters
            for p, pic in enumerate(cluster):  
                clustercount[k] +=1            #Count number of pictures in the cluster
                for l,pixel in enumerate(pic):
                    valuecountmatrix[k][int(pixel*16)][l] +=1
                    attributesums[k][l] += pixel  #Sum all pixel values for each feature
        
        for c, classrow in enumerate(attributesums):
            for s, pixelsum in enumerate(classrow):
                if clustercount[c] != 0:
                    self.means[c][s] = pixelsum/clustercount[c]
                else:
                    self.means[c][s] = 0
        
        #For every class calculate variance
        self.CovK = np.zeros([10,len(features[1])])
        for cluster in range(10):
            for i, pixelsums in enumerate(valuecountmatrix[cluster]):
                for j,sum in enumerate(pixelsums):
                    self.CovK[cluster][j] += j*(i-self.means[cluster][j])**2   
                    
        epsilon = 0.1
        for cluster in range(10):
            self.CovK[cluster] = (1/clustercount[cluster])*(self.CovK[cluster])+epsilon
        return
    
        #End of initialization. CovK, Mu_k and Pi_k are thus created
        
    def fit(self, features, k = 10, r = 20):
        
        Clustered = False
        #Start the iteration
        count = 1
        while Clustered == False:
            print('Iteration: ' + str(count))
            count+=1
            
            
            r = np.zeros([len(features), k])
            for i, picture in enumerate(features):
                expected = np.zeros(k)
                for cluster in range(10):
                    dist = (picture-self.means[cluster])**2
                    probj = (1/(np.sqrt(2*3.14159)*np.sqrt(self.CovK[cluster])))*np.exp(-(dist)/(2*self.CovK[cluster]))
                    probj = np.prod(probj)*self.pi_k[cluster]
                    expected[cluster] = probj
                ri = expected/sum(expected)
                r[i] = ri
                
            #rik has been computed for all images i and clusters k.

            #M-step
            N = len(features)
            #Compute for each class the sum of all ri
            rk = np.zeros(k)
            for j in range(10):
                for i in range(len(features)):      
                    rk[j] +=  r[i][j]
                self.pi_k[j] = rk[j]/N
            #Compute for each class the new pi_k
            
            normsum = 0

            #Update the means and the variances
            for l in range(10):
                mean = 0
                Cov = 0       
                for i in range(len(features)):
                    Cov += r[i][l]*(features[i]*np.transpose(features[i]))
                    mean += r[i][l]*features[i]
                mean = mean/rk[l]
                Cov = Cov/rk[l] - mean*np.transpose(mean) + 0.01
                normsum += norm(self.means[l]-mean)
                self.means[l] = mean
                self.CovK[l] = Cov
            print('Change in L2-norms: ' +str(normsum))
            if normsum < 0.01:
                Clustered = True
                                    
    def predict(self, data):
        predicted = []
        
        #For every picture in the data:
        for pictures in data:
            pred = 1000
            predicted_probability = 0
            
            for i in range(10):
                cond_prob = 1
                for p, pixel in enumerate(pictures):
                        prob = (1/(np.sqrt(2*3.14159)*np.sqrt(self.CovK[i][p])))*np.exp(-((pixel-self.means[i][p])**2)/(2*self.CovK[i][p]))                               
                        cond_prob = cond_prob*prob
                prob_y = self.pi_k[i]*cond_prob
                if prob_y > predicted_probability:
                    predicted_probability = prob_y
                    pred = i
            predicted.append(pred)
    
        return predicted
        

In [4]:
d = digits_train_features/16
EM = ExpectationMaximisation(d)


In [5]:
EM.fit(d)

Iteration: 1
Change in L2-norms: 3.749708291060642
Iteration: 2
Change in L2-norms: 0.07000741203680746
Iteration: 3
Change in L2-norms: 0.669725726564204
Iteration: 4
Change in L2-norms: 3.598623658956015
Iteration: 5
Change in L2-norms: 5.219956224919757
Iteration: 6
Change in L2-norms: 4.476642467072828
Iteration: 7
Change in L2-norms: 2.61113947831729
Iteration: 8
Change in L2-norms: 1.3533971164913121
Iteration: 9
Change in L2-norms: 1.026729346064542
Iteration: 10
Change in L2-norms: 1.078182777861262
Iteration: 11
Change in L2-norms: 1.023293480299798
Iteration: 12
Change in L2-norms: 0.5978002865587844
Iteration: 13
Change in L2-norms: 0.49726937436171403
Iteration: 14
Change in L2-norms: 0.3808498101329075
Iteration: 15
Change in L2-norms: 0.35141365692727733
Iteration: 16
Change in L2-norms: 0.37982761438178325
Iteration: 17
Change in L2-norms: 0.25159335398324056
Iteration: 18
Change in L2-norms: 0.17565815669091037
Iteration: 19
Change in L2-norms: 0.14674672757255441
Itera

In [6]:
 pred = EM.predict(d)

# Evaluation of results and comparison with Sklearn KMeans


In [7]:
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=10)
clusters = clustering.fit(d)

prediction = clustering.predict(d)

print('SKlearn: ')
print('Confusion matrix')
print()
print(metrics.confusion_matrix(digits_train_labels, prediction))
print()
print('Completeness score against ground truth: ' + str(metrics.completeness_score(digits_train_labels, prediction)))
print('Homogeneity score against ground truth: ' + str(metrics.homogeneity_score(digits_train_labels, prediction)))
print('Adjusted mutual info against ground truth: ' + str(metrics.adjusted_mutual_info_score(digits_train_labels, prediction)))
print()
print('EM-Algorithm')
print()
print('Confusion matrix')
print(metrics.confusion_matrix(digits_train_labels, pred))
print()
print('Completeness score against ground truth: ' + str(metrics.completeness_score(digits_train_labels, pred)))
print('Homogeneity score against ground truth: ' + str(metrics.homogeneity_score(digits_train_labels, pred)))
print('Adjusted mutual info against ground truth: ' + str(metrics.adjusted_mutual_info_score(digits_train_labels, pred)))
print()
print('EM against Kmeans:')
print('Completeness score against Kmeans ' + str(metrics.completeness_score(pred, prediction)))
print('Homogeneity score against Kmeans: ' + str(metrics.homogeneity_score(pred, prediction)))
print('Adjusted mutual info against Kmeans: ' + str(metrics.adjusted_mutual_info_score(pred, prediction)))


SKlearn: 
Confusion matrix

[[  0   0   0 125   0   0   0   0   0   0]
 [  2  63   0   0   0  25   0   0   0  39]
 [  0   8   2   0   2 108   0   0   1   3]
 [  0   0 110   0   2   0   0   2  16   0]
 [  0   2   0   0   7   0 109   0   0   6]
 [  1   0   1   0   0   0   1  94  29   0]
 [124   2   0   1   0   0   0   0   0   0]
 [  0   0   0   0 122   0   0   1   0   2]
 [  1  61   1   0   1   2   0   8  44   4]
 [  0   0   3   0   7   0   0   3  97  15]]

Completeness score against ground truth: 0.7551388129562558
Homogeneity score against ground truth: 0.7466873334922608
Adjusted mutual info against ground truth: 0.747266298787389

EM-Algorithm

Confusion matrix
[[  0   0   1   1 123   0   0   0   0   0]
 [ 65   0   0   0   0  37   0  27   0   0]
 [ 13   0   0   0   0   1   0  94  16   0]
 [  0   0  17   0   0   1   3   3 106   0]
 [  0   0   0 110   0   5   8   0   0   1]
 [  0   0  31   1   0   0   1   0   1  92]
 [  2 122   3   0   0   0   0   0   0   0]
 [  0   0   0   0   0  17 1

# Discarded code


In [ ]:

class ExpectationMaximisation:
    
    def __init__(self, features, k = 10):
        
        #Initialise pi_k, mu_k, sum_k where pi_k is class prior, mu_k is means for attribute value j in class k
        self.CovK = 1
        self.means = 1
        self.pi_k = np.ones([10,1])*1/10
        self.clustered = []

        #Make random subsets for initialization
        number_of_samples = len(features)
        subset_size = number_of_samples/k
        
        for i in range(k):
            self.clustered.append((features[round(i*subset_size):round((i+1)*subset_size)]))  
        
        #Find the number of unique values in features. 17 for digits, 3 for sumdigits
        uniqueval = 0
        uniquelist = []
        for pic in features:
            for pixel in pic:
                if pixel not in uniquelist:
                    uniquelist.append(pixel)
        uniqueval = len(uniquelist)
        
        attributesums = np.ones([k,len(features[0])])*0.1
        clustercount = np.zeros(k)
        self.means = np.zeros([k,len(features[0])])  # Means for all features in cluster k
        
        valuecountmatrix = np.zeros([k,uniqueval,len(features[1])])
        for k,cluster in enumerate(self.clustered): #Iterate through clusters
            for p, pic in enumerate(cluster):  
                clustercount[k] +=1            #Count number of pictures in the cluster
                for l,pixel in enumerate(pic):
                    valuecountmatrix[k][int(pixel)][l] +=1
                    attributesums[k][l] += pixel  #Sum all pixel values for each feature
        
        for c, classrow in enumerate(attributesums):
            for s, pixelsum in enumerate(classrow):
                if clustercount[c] != 0:
                    self.means[c][s] = pixelsum/clustercount[c]
                else:
                    self.means[c][s] = 0
        
        #For every class calculate variance
        self.CovK = np.zeros([10,len(features[1])])
        for cluster in range(10):
            for i, pixelsums in enumerate(valuecountmatrix[cluster]):
                for j,sum in enumerate(pixelsums):
                    self.CovK[cluster][j] += j*(i-self.means[cluster][j])**2   
                    
        epsilon = 0.1
        for cluster in range(10):
            self.CovK[cluster] = (1/clustercount[cluster])*(self.CovK[cluster])+epsilon
        return
    
        #End of initialization. CovK, Mu_k and Pi_k are thus created
        
    def fit(self, features, k = 10, r = 20):
        
        #Start the iteration
        
        for r in range(r):
            print('BEGINNING ITERATION: ' + str(r))
            
            
            #E-step.
            #For every picture in the data compute rik
            #rik is probability of picture i belonging to cluster k assuming feature distributions j for kluster k,
            #divided by sum for all clusters
            r = np.zeros([len(features), k])
            for i,picture in enumerate(features):
                expected = np.zeros(k)
                for cluster in range(k):
                    nominator = self.pi_k[cluster]
                    for j, pixel in enumerate(picture):
                        prob = (1/(np.sqrt(2*3.14159)*np.sqrt(self.CovK[cluster][j])))*math.exp(-((pixel-self.means[cluster][j])**2)/(2*self.CovK[cluster][j]))                                  
                        nominator = nominator*prob
                    expected[cluster] = nominator
                ri = expected/sum(expected)
                r[i] = ri
            #rik has been computed for all images i and clusters k.
            print('E-STEP DONE')

            #M-step
            N = len(features)
            #Compute for each class the sum of all ri
            rk = np.zeros(k)
            for j in range(10):
                for i in range(len(features)):      
                    rk[j] +=  r[i][j]
                self.pi_k[j] = rk[j]/N
            #Compute for each class the new pi_k

            #Update the means and the variances
            for l in range(10):
                mean = 0
                Cov = 0       
                for i in range(len(features)):
                    Cov += r[i][l]*(features[i]*np.transpose(features[i]))
                    mean += r[i][l]*features[i]
                mean = mean/rk[l]
                Cov = Cov/rk[l] - mean*np.transpose(mean) + 0.01
                self.means[l] = mean
                self.CovK[l] = Cov
            print('M-STEP DONE')
                                    
    def predict(self, data):
        predicted = []
        
        #For every picture in the data:
        for pictures in data:
            pred = 1000
            predicted_probability = 0
            
            for i in range(10):
                cond_prob = 1
                for p, pixel in enumerate(pictures):
                        prob = (1/(np.sqrt(2*3.14159)*np.sqrt(self.CovK[i][p])))*math.exp(-((pixel-self.means[i][p])**2)/(2*self.CovK[i][p]))                               
                        cond_prob = cond_prob*prob
                prob_y = self.pi_k[i]*cond_prob
                if prob_y > predicted_probability:
                    predicted_probability = prob_y
                    pred = i
            predicted.append(pred)
    
        return predicted
        
